In [4]:
from transformers import AutoTokenizer, BertForSequenceClassification
import numpy as np
import matplotlib.pyplot as plt
from Teste_extracao import extract_sentences_from_pdf

In [5]:
pred_mapper = {
    0: "POSITIVE",
    1: "NEGATIVE",
    2: "NEUTRAL"
}
tokenizer = AutoTokenizer.from_pretrained("lucas-leme/FinBERT-PT-BR")
finbertptbr = BertForSequenceClassification.from_pretrained("lucas-leme/FinBERT-PT-BR")

In [6]:
import fitz  # PyMuPDF
import re
import csv  # Importe o módulo csv
from datetime import datetime
import locale
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

def preproc(path):
	file = path
	### Analise
	# Abra o arquivo PDF
	doc = fitz.open(file)

	# Extrair texto da página desejada (por exemplo, a primeira página)
	page = doc[0]

	text = page.get_text()

	lines = text.split('\n')

	# Verifique se há pelo menos 8 linhas e acesse a oitava linha para extrair a data
	if len(lines) >= 8:
		data_corr = lines[6]
		data_correspondente = data_corr.replace("º", "")
		print(data_correspondente)
		data_match = re.search(r'\d+\s+de\s+[a-zA-Z]+\s+de\s+\d{4}', data_correspondente)
		print(data_match)
		if data_match:
			data_texto = data_match.group()
			data_formatada = datetime.strptime(data_texto, "%d de %B de %Y").strftime("%Y-%m-%d")
			print("Data:", data_formatada)
		else:
			data_corr = lines[8]
			data_correspondente = data_corr.replace("º", "")
			print(data_correspondente)
			data_match = re.search(r'\d+\s+de\s+[a-zA-Z]+\s+de\s+\d{4}', data_correspondente)
			print(data_match)
			if data_match:
				data_texto = data_match.group()
				data_formatada = datetime.strptime(data_texto, "%d de %B de %Y").strftime("%Y-%m-%d")
				print("Data:", data_formatada)
	else:
		print("Não há informações suficientes nas linhas do PDF para extrair a data da sétima linha.")

	#sentencas = extract_sentences_from_pdf(file)[701:779]
	sentencas = extract_sentences_from_pdf(file)
	tokens = tokenizer(sentencas , return_tensors="pt",
					padding=True, truncation=True, max_length=512)
	finbertptbr_outputs = finbertptbr(**tokens)

	preds = [pred_mapper[np.argmax(pred)] for pred in finbertptbr_outputs.logits.cpu().detach().numpy()]
	print(preds)

	Posi = 0
	Neg = 0
	Neutral = 0
	for t in preds:
		if t =="POSITIVE":
			Posi+=1
		elif t == "NEGATIVE":
			Neg+=1
		else:
			Neutral+=1
	print(Posi, Neutral, Neg)

	indice =((Posi - Neg) / (Posi + Neg + Neutral))
	# Salvar os resultados em um arquivo CSV
	with open("resultados.csv", mode="a", newline="") as file:
		writer = csv.writer(file)
		#writer.writerow(["Data da reuniao", "positivo", "negativo", "neutro", "indice"])
		writer.writerow([data_formatada, Posi, Neg, Neutral, indice])


	print("Resultados salvos em 'resultados.csv'.")


In [7]:
'''for trimestre in [ "4T15"]:
    file_path = f"G:\\Meu Drive\\IC - NLP\\Textos perguntas e respostas de resultados\\Itau - 2013-2023\\transcrição do áudio {trimestre}.pdf"
    preproc(file_path)'''
preproc("Transcricao_do_audio1T23.pdf")

O evento de hoje será dividido em duas partes. Na primeira, o Milton explicará a nossa 
None
de perguntas e respostas, em que analistas e investidores poderão interagir diretamente 
None



KeyboardInterrupt



In [ ]:
#preproc("G:\\Meu Drive\\IC - NLP\\Textos perguntas e respostas de resultados\\Itau - 2013-2023\\transcrição do áudio 1T16.pdf")
#preproc("G:\\Meu Drive\\IC - NLP\\Textos perguntas e respostas de resultados\\Itau - 2013-2023\\transcrição do áudio 2T16.pdf")
#preproc("G:\\Meu Drive\\IC - NLP\\Textos perguntas e respostas de resultados\\Itau - 2013-2023\\transcrição do áudio 2T17.pdf")
preproc("G:\\Meu Drive\\IC - NLP\\Textos perguntas e respostas de resultados\\Itau - 2013-2023\\transcrição do áudio 3T18.pdf")
#preproc("G:\\Meu Drive\\IC - NLP\\Textos perguntas e respostas de resultados\\Itau - 2013-2023\\transcrição do áudio 2T19.pdf")
#preproc("G:\\Meu Drive\\IC - NLP\\Textos perguntas e respostas de resultados\\Itau - 2013-2023\\transcrição do áudio 3T19.pdf")
#preproc("G:\\Meu Drive\\IC - NLP\\Textos perguntas e respostas de resultados\\Itau - 2013-2023\\transcrição do áudio 4T19.pdf")

30 de outubro de 2018 
<re.Match object; span=(0, 21), match='30 de outubro de 2018'>
Data: 2018-10-30
['NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEUTRAL', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEUTRAL', 'NEGATIVE', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'POSITIVE', 'NEUTRAL', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEUTRAL', 'POSITIVE', 'NEUTRAL', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEUTRAL', 'POSITIVE'

In [ ]:
sentencas = extract_sentences_from_pdf("Transcricao_do_audio1T23.pdf")[701:779]
tokens = tokenizer(sentencas , return_tensors="pt",
                   padding=True, truncation=True, max_length=512)
finbertptbr_outputs = finbertptbr(**tokens)

In [ ]:
preds = [pred_mapper[np.argmax(pred)] for pred in finbertptbr_outputs.logits.cpu().detach().numpy()]
print(preds)

In [ ]:
Posi = 0
Neg = 0
Neutral = 0
for t in preds:
    if t =="POSITIVE":
        Posi+=1
    elif t == "NEGATIVE":
        Neg+=1
    else:
        Neutral+=1
print(Posi, Neutral, Neg)

2 2 1


In [ ]:
import csv

Posi = 0
Neg = 0
Neutral = 0
preds = ["POSITIVE", "NEUTRAL", "POSITIVE", "NEGATIVE", "NEUTRAL"]  # Exemplo de valores de preds

for t in preds:
    if t == "POSITIVE":
        Posi += 1
    elif t == "NEGATIVE":
        Neg += 1
    else:
        Neutral += 1

print("Positivos:", Posi)
print("Neutros:", Neutral)
print("Negativos:", Neg)

indice =((Posi - Neg) / (Posi + Neg + Neutral))
# Salvar os resultados em um arquivo CSV
with open("resultados.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Data da reuniao", "positivo", "negativo", "neutro", "indice"])
    writer.writerow([data_formatada, Posi, Neg, Neutral, indice])


print("Resultados salvos em 'resultados.csv'.")


Positivos: 2
Neutros: 2
Negativos: 1
Resultados salvos em 'resultados.csv'.


In [ ]:
import fitz  # PyMuPDF
import re
from datetime import datetime
import locale
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')


# Abra o arquivo PDF
doc = fitz.open("G:\\Meu Drive\\IC - NLP\\Textos perguntas e respostas de resultados\\Itau - 2013-2023\\transcrição do áudio 2T19.pdf")

# Extrair texto da página desejada (por exemplo, a primeira página)
page = doc[0]

text = page.get_text()

lines = text.split('\n')

# Verifique se há pelo menos 8 linhas e acesse a oitava linha para extrair a data
if len(lines) >= 8:
    data_correspondente = lines[6]
    data_match = re.search(r'\d+\s+de\s+[a-zA-Z]+\s+de\s+\d{4}', data_correspondente)

    if data_match:
        data_texto = data_match.group()
        data_formatada = datetime.strptime(data_texto, "%d de %B de %Y").strftime("%Y-%m-%d")
        print("Data:", data_formatada)
    else:
        print("Data não encontrada na sétima linha.")
else:
    print("Não há informações suficientes nas linhas do PDF para extrair a data da sétima linha.")


Data: 2019-07-29
